# Testing Accuracy with Batches

For each user that we test, group into batches with one sample per direction.

Test each sample individually, but average the prediction across the entire batch. 

## Code

In [1]:
import os
from random import randint
import numpy as np
import pandas as pd
from keras.layers import Dense, Activation, Input, CuDNNLSTM, Bidirectional, Dropout
from keras.models import Model, Sequential, load_model
from keras.preprocessing.sequence import pad_sequences
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder, StandardScaler

Using TensorFlow backend.


### Data Preperation

In [2]:
# Return a list of usernames based on the file names in feature files. 
def get_usernames(train_files):
    usernames = []
    for f in train_files:
        ext = f[-4:]
        if(ext == '.csv'):
            username = f[:f.index('-')]
            usernames.append(username)
    return usernames


# Return a list of Pandas Dataframes from the csv feature files.  
def getdfs(path, files):
    dfs = []
    for f in files: 
        ext = f[-4:]
        if(ext == '.csv'):
            file = os.path.join(path, f)
            df = pd.read_csv(file)
            dfs.append(df)
    return dfs


# Create sequences out of a dataframe by splitting the sample number into a 2D array.
# For each sample in a given direction: create a list of the 10 features, and append that list to the sample
# Remove the sample number from the features as this is not important to the classifier. 
    '''
    Goal: [Sample 1: [[10 features], [10 features], [10 features]], Sample 2: [[10 features], [10 features], [10 features]]]
    '''
def create_sequences(df):

    samples = df['Sample'].unique()
    dirs = df['Direction'].unique()
    sequences = []
    for dir in dirs:
        dir_frame = df.loc[df['Direction'] == dir]
        for sample in samples:
            sample_frame = dir_frame.loc[dir_frame['Sample'] == sample]
            sf = sample_frame.drop(columns=['Sample']) # Remove the sample number before adding to the list
            values = list(sf.values.tolist())
            if len(values) > 0:
                sequences.append(values)
    return sequences


# encode each of the features with a one hot encoding for the direction variable
def encode(x):
    newx = []
    for sequence in x:
        new_seq = []
        for feature in sequence:
            new_seq.append(one_hot_encode(feature))
        newx.append(new_seq)
    return newx


# One hot encode the direction variable only.
# The direction is the first variable in the feature vec, so base the one hot encoding on this. 
# If the direction value is 0 (used for padding only) then use all zeros not an encoding
def one_hot_encode(feature_vec):
    feature_vec = list(feature_vec)
    zeros = [0, 0, 0, 0, 0, 0, 0, 0]
    n_labels = 8
    i = np.eye(n_labels)
    
    dir = feature_vec[0]
    if dir < 1:
        enc = zeros
    else:
        enc = i[int(feature_vec[0]) - 1]

    feature_vec.remove(feature_vec[0])

    feature_vec[0:0] = enc
    return feature_vec


def generate_batches(username, usernames, dfs):
    pass
    
# Create a test set similar to the create_test_set method
# This version uses ALL test data for every user, not just a subset. 
def create_full_test(username, usernames, dfs):
    # Create the positive examples. Set y to 1
    useridx = usernames.index(username)
    user_df = dfs[useridx]
    pos_batches = batchify(user_df, is_pos=True)
    ys = [1 for _ in pos_batches]
    
    # Create the negative examples
    neg_batches = []
    for user in usernames:
        if user != username:
            idx = usernames.index(user)
            df = dfs[idx]
            user_batches = batchify(df, is_pos=True)
            neg_batches += user_batches
    
    batches = pos_batches + neg_batches
    ys += [0 for _ in neg_batches]
    
    x = []
    for batch in batches:
        x.append(encode(pad_sequences(batch, maxlen=50, dtype='float32', value=[0 for x in range(10)])))
    X = np.asarray(x)
    Y = np.asarray(ys)               
    return X, Y

def create_test_set(username, usernames, dfs):
    
    # Create the positive examples. Set y to 1
    useridx = usernames.index(username)
    user_df = dfs[useridx]
    pos_batches = batchify(user_df, is_pos=True)
    ys = [1 for _ in pos_batches]
    
    # Create the negative examples
    neg_batches = []
    for user in usernames:
        if user != username:
            idx = usernames.index(user)
            df = dfs[idx]
            user_batches = batchify(df, is_pos=False)
            neg_batches += user_batches
    
    batches = pos_batches + neg_batches
    ys += [0 for _ in neg_batches]
    
    x = []
    for batch in batches:
        x.append(encode(pad_sequences(batch, maxlen=50, dtype='float32', value=[0 for x in range(10)])))
    X = np.asarray(x)
    Y = np.asarray(ys)               
    return X, Y
 


def batchify(dataframe, is_pos=True):
    dirs = [1,2,3,4,5,6,7,8]
    direction_seqs = [] 
    for i in dirs:
        dir_i = dataframe.loc[dataframe['Direction'] == i]
        seqs = create_sequences(dir_i)
        
        if not is_pos:
            # Randomly choose 4 samples for each direction for a user.
            seqs = [seqs[randint(0, len(seqs) -1)] for x in range(4)]
        direction_seqs.append(seqs)

    # Batch 1 sample per direction into a batch
    batches = []
    if not is_pos:
        num_batches = 4
    else:
        num_batches = find_num_batches(direction_seqs)
    
    for count in range(num_batches):
        batch = []
        for dir in direction_seqs:
            batch.append(dir[count])
        batches.append(batch)
    
    return batches

        
# Find the number of batches in a series. 
def find_num_batches(seqs):
    lens = [len(s) for s in seqs]
    return min(lens)




### Model Testing

In [3]:
# convert the probabilities to predictions on a threshold
def convert(p, threshold):
    return 1 if p >= threshold else 0


# Given a set of probabilities and true Y-Values, 
# Convert probabilities to predictions. 
# Determine accuracy and return dictionary with tracked metrics. 
def calc_accuracy_metrics(y_preds, y_actual):
    pairs = list(zip(y_preds, y_actual))
    
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for pair in pairs:
        if pair[0] == 1:
            if pair[1] == 1:
                tp += 1
            else:
                fp += 1
        else:
            if pair[1] == 0:
                tn += 1
            else:
                fn += 1
    
    try:
        far = fp / (fp + tn)
    except ZeroDivisionError:
        far = -1
    try:    
        frr = fn / (fn + tp)
    except ZeroDivisionError:
        frr = -1
    try:    
        precision = tp / (tp + fp)
    except ZeroDivisionError:
        precision = -1
    try:
        recall = tp / (tp + fn)
    except ZeroDivisionError:
        recall = -1
    try:    
        f1_score = (2 * precision * recall) / (precision + recall)
    except ZeroDivisionError:
        f1_score = -1
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    
    performance_results = {'far': far, 'frr': frr, 'precision': precision, 'recall': recall, 'f1': f1_score, 
                           'accuracy': accuracy}
    return performance_results


# Get the predicitons from the classifier. 
# Returns a list of predictions. 
def get_predictions(clf, test_x):
    ypreds = clf.predict(test_x, batch_size=10, verbose=False)
    return ypreds


def test_batches(clf, test_x, thresh):
    batch_preds = []
    for batch in test_x:
        y_preds = get_predictions(clf, batch)
        yhats = [convert(y, thresh) for y in y_preds]
        prediction = round(sum(yhats)/len(yhats))
        batch_preds.append(prediction)
    return batch_preds


def test_overall_acc(accuracy_dicts):
    far = 0
    frr = 0
    precision = 0
    recall = 0
    f1_score = 0
    accuracy = 0

    for d in accuracy_dicts:
        far += d.get('far')
        frr += d.get('frr')
        precision += d.get('precision')
        recall += d.get('recall')
        f1_score += d.get('f1')
        accuracy += d.get('accuracy')

    print('Overall Accuracy - Average of Individual Models:\n')
    print('FAR : {}'.format(far / len(usernames)))
    print('FRR : {}'.format(frr / len(usernames)))
    print('Precision : {}'.format(precision / len(usernames)))
    print('Recall : {}'.format(recall / len(usernames)))
    print('F1-Score : {}'.format(f1_score / len(usernames)))
    print('Accuracy : {}'.format(accuracy / len(usernames)))

### Create Test Sets for each user

In [4]:
test_path = os.path.join(os.getcwd(), 'RNN-Test-Aligned')
model_save_path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
test_files = os.listdir(test_path)

usernames = get_usernames(test_files)
test_dfs = getdfs(test_path, test_files)

testing_xs = []
testing_ys = []
for user in usernames:
    xtest, ytest = create_test_set(user, usernames, test_dfs)
    testing_xs.append(xtest)
    testing_ys.append(ytest)

## Test The Models

### Looking at each user individually to update thresholds

#### Amanda11
Best Threshold EER: 0.16  F: 0.2

In [7]:
username = usernames[0]
test_x = testing_xs[0]
test_y = testing_ys[0]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(10, 20):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR AMANDA11
Threshold 0.1
far: 0.07258064516129033
frr: 0.0
precision: 0.5263157894736842
recall: 1.0
f1: 0.6896551724137931
accuracy: 0.9328358208955224

Threshold 0.11
far: 0.07258064516129033
frr: 0.0
precision: 0.5263157894736842
recall: 1.0
f1: 0.6896551724137931
accuracy: 0.9328358208955224

Threshold 0.12
far: 0.06451612903225806
frr: 0.0
precision: 0.5555555555555556
recall: 1.0
f1: 0.7142857142857143
accuracy: 0.9402985074626866

Threshold 0.13
far: 0.06451612903225806
frr: 0.0
precision: 0.5555555555555556
recall: 1.0
f1: 0.7142857142857143
accuracy: 0.9402985074626866

Threshold 0.14
far: 0.06451612903225806
frr: 0.0
precision: 0.5555555555555556
recall: 1.0
f1: 0.7142857142857143
accuracy: 0.9402985074626866

Threshold 0.15
far: 0.04838709677419355
frr: 0.0
precision: 0.625
recall: 1.0
f1: 0.7692307692307693
accuracy: 0.9552238805970149

Threshold 0.16
far: 0.04032258064516129
frr: 0.0
precision: 0.6666666666666666
recall: 1.0
f1: 0.8
accuracy: 0.9626865671641791


#### Amelia1
Best Threshold 0.15 Both

In [10]:
username = usernames[1]
test_x = testing_xs[1]
test_y = testing_ys[1]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(10, 20):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR AMELIA1
Threshold 0.1
Model 1
far: 0.07258064516129033
frr: 0.14285714285714285
precision: 0.4
recall: 0.8571428571428571
f1: 0.5454545454545455
accuracy: 0.9236641221374046

Threshold 0.11
Model 1
far: 0.06451612903225806
frr: 0.14285714285714285
precision: 0.42857142857142855
recall: 0.8571428571428571
f1: 0.5714285714285714
accuracy: 0.9312977099236641

Threshold 0.12
Model 1
far: 0.06451612903225806
frr: 0.14285714285714285
precision: 0.42857142857142855
recall: 0.8571428571428571
f1: 0.5714285714285714
accuracy: 0.9312977099236641

Threshold 0.13
Model 1
far: 0.056451612903225805
frr: 0.14285714285714285
precision: 0.46153846153846156
recall: 0.8571428571428571
f1: 0.6
accuracy: 0.9389312977099237

Threshold 0.14
Model 1
far: 0.056451612903225805
frr: 0.14285714285714285
precision: 0.46153846153846156
recall: 0.8571428571428571
f1: 0.6
accuracy: 0.9389312977099237

Threshold 0.15
Model 1
far: 0.056451612903225805
frr: 0.14285714285714285
precision: 0.46153846153846156

#### Andy1
Best Threshold EER: 0.1 F: 0.73

In [12]:
username = usernames[2]
test_x = testing_xs[2]
test_y = testing_ys[2]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(70, 80):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR ANDY1
Threshold 0.7000000000000001
Model 1
far: 0.016129032258064516
frr: 0.14285714285714285
precision: 0.75
recall: 0.8571428571428571
f1: 0.7999999999999999
accuracy: 0.9770992366412213

Threshold 0.71
Model 1
far: 0.016129032258064516
frr: 0.14285714285714285
precision: 0.75
recall: 0.8571428571428571
f1: 0.7999999999999999
accuracy: 0.9770992366412213

Threshold 0.72
Model 1
far: 0.0
frr: 0.14285714285714285
precision: 1.0
recall: 0.8571428571428571
f1: 0.923076923076923
accuracy: 0.9923664122137404

Threshold 0.73
Model 1
far: 0.0
frr: 0.14285714285714285
precision: 1.0
recall: 0.8571428571428571
f1: 0.923076923076923
accuracy: 0.9923664122137404

Threshold 0.74
Model 1
far: 0.0
frr: 0.14285714285714285
precision: 1.0
recall: 0.8571428571428571
f1: 0.923076923076923
accuracy: 0.9923664122137404

Threshold 0.75
Model 1
far: 0.0
frr: 0.14285714285714285
precision: 1.0
recall: 0.8571428571428571
f1: 0.923076923076923
accuracy: 0.9923664122137404

Threshold 0.76
Model 1


#### Andy21 *Bad Performance from this user*
Best Threshold 0.009 both

In [15]:
username = usernames[3]
test_x = testing_xs[3]
test_y = testing_ys[3]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(1, 11):
    thresh = 0.001 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR ANDY21
Threshold 0.001
Model 1
far: 0.7338709677419355
frr: 0.0
precision: 0.021505376344086023
recall: 1.0
f1: 0.04210526315789474
accuracy: 0.2777777777777778

Threshold 0.002
Model 1
far: 0.2661290322580645
frr: 0.0
precision: 0.05714285714285714
recall: 1.0
f1: 0.1081081081081081
accuracy: 0.7380952380952381

Threshold 0.003
Model 1
far: 0.1532258064516129
frr: 0.0
precision: 0.09523809523809523
recall: 1.0
f1: 0.17391304347826084
accuracy: 0.8492063492063492

Threshold 0.004
Model 1
far: 0.12096774193548387
frr: 0.0
precision: 0.11764705882352941
recall: 1.0
f1: 0.21052631578947367
accuracy: 0.8809523809523809

Threshold 0.005
Model 1
far: 0.08064516129032258
frr: 0.0
precision: 0.16666666666666666
recall: 1.0
f1: 0.2857142857142857
accuracy: 0.9206349206349206

Threshold 0.006
Model 1
far: 0.06451612903225806
frr: 0.0
precision: 0.2
recall: 1.0
f1: 0.33333333333333337
accuracy: 0.9365079365079365

Threshold 0.007
Model 1
far: 0.04838709677419355
frr: 0.0
precision: 0

#### Benjamin1
Best Threshold EER 0.16 F 0.3

In [17]:
username = usernames[4]
test_x = testing_xs[4]
test_y = testing_ys[4]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(10, 20):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()

ACCURACY FOR BENJAMIN1
Threshold 0.1
far: 0.06451612903225806
frr: 0.0
precision: 0.5
recall: 1.0
f1: 0.6666666666666666
accuracy: 0.9393939393939394

Threshold 0.11
far: 0.06451612903225806
frr: 0.0
precision: 0.5
recall: 1.0
f1: 0.6666666666666666
accuracy: 0.9393939393939394

Threshold 0.12
far: 0.06451612903225806
frr: 0.0
precision: 0.5
recall: 1.0
f1: 0.6666666666666666
accuracy: 0.9393939393939394

Threshold 0.13
far: 0.06451612903225806
frr: 0.0
precision: 0.5
recall: 1.0
f1: 0.6666666666666666
accuracy: 0.9393939393939394

Threshold 0.14
far: 0.056451612903225805
frr: 0.0
precision: 0.5333333333333333
recall: 1.0
f1: 0.6956521739130436
accuracy: 0.946969696969697

Threshold 0.15
far: 0.04838709677419355
frr: 0.0
precision: 0.5714285714285714
recall: 1.0
f1: 0.7272727272727273
accuracy: 0.9545454545454546

Threshold 0.16
far: 0.04838709677419355
frr: 0.0
precision: 0.5714285714285714
recall: 1.0
f1: 0.7272727272727273
accuracy: 0.9545454545454546

Threshold 0.17
far: 0.04838709

#### Blair1
Best Threshold 0.05 Both

In [19]:
username = usernames[5]
test_x = testing_xs[5]
test_y = testing_ys[5]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(1, 10):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR BLAIR1
Threshold 0.01
Model 1
far: 0.08870967741935484
frr: 0.0
precision: 0.5217391304347826
recall: 1.0
f1: 0.6857142857142856
accuracy: 0.9191176470588235

Threshold 0.02
Model 1
far: 0.04838709677419355
frr: 0.0
precision: 0.6666666666666666
recall: 1.0
f1: 0.8
accuracy: 0.9558823529411765

Threshold 0.03
Model 1
far: 0.04838709677419355
frr: 0.0
precision: 0.6666666666666666
recall: 1.0
f1: 0.8
accuracy: 0.9558823529411765

Threshold 0.04
Model 1
far: 0.03225806451612903
frr: 0.08333333333333333
precision: 0.7333333333333333
recall: 0.9166666666666666
f1: 0.8148148148148148
accuracy: 0.9632352941176471

Threshold 0.05
Model 1
far: 0.024193548387096774
frr: 0.08333333333333333
precision: 0.7857142857142857
recall: 0.9166666666666666
f1: 0.8461538461538461
accuracy: 0.9705882352941176

Threshold 0.06
Model 1
far: 0.024193548387096774
frr: 0.16666666666666666
precision: 0.7692307692307693
recall: 0.8333333333333334
f1: 0.8
accuracy: 0.9632352941176471

Threshold 0.07
Mod

#### Blake1
Best Threshold 0.02 Both

In [21]:
username = usernames[6]
test_x = testing_xs[6]
test_y = testing_ys[6]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(1, 10):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR BLAKE1
Threshold 0.01
Model 1
far: 0.0967741935483871
frr: 0.0
precision: 0.45454545454545453
recall: 1.0
f1: 0.625
accuracy: 0.9104477611940298

Threshold 0.02
Model 1
far: 0.04838709677419355
frr: 0.0
precision: 0.625
recall: 1.0
f1: 0.7692307692307693
accuracy: 0.9552238805970149

Threshold 0.03
Model 1
far: 0.03225806451612903
frr: 0.3
precision: 0.6363636363636364
recall: 0.7
f1: 0.6666666666666666
accuracy: 0.9477611940298507

Threshold 0.04
Model 1
far: 0.024193548387096774
frr: 0.3
precision: 0.7
recall: 0.7
f1: 0.7
accuracy: 0.9552238805970149

Threshold 0.05
Model 1
far: 0.024193548387096774
frr: 0.3
precision: 0.7
recall: 0.7
f1: 0.7
accuracy: 0.9552238805970149

Threshold 0.06
Model 1
far: 0.024193548387096774
frr: 0.3
precision: 0.7
recall: 0.7
f1: 0.7
accuracy: 0.9552238805970149

Threshold 0.07
Model 1
far: 0.024193548387096774
frr: 0.4
precision: 0.6666666666666666
recall: 0.6
f1: 0.631578947368421
accuracy: 0.9477611940298507

Threshold 0.08
Model 1
far: 0

#### Chloe1
Best Threshold EER: 0.03 F: 0.07

In [23]:
username = usernames[7]
test_x = testing_xs[7]
test_y = testing_ys[7]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(1, 10):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR CHLOE1
Threshold 0.01
Model 1
far: 0.12903225806451613
frr: 0.0
precision: 0.36
recall: 1.0
f1: 0.5294117647058824
accuracy: 0.8796992481203008

Threshold 0.02
Model 1
far: 0.07258064516129033
frr: 0.0
precision: 0.5
recall: 1.0
f1: 0.6666666666666666
accuracy: 0.9323308270676691

Threshold 0.03
Model 1
far: 0.04032258064516129
frr: 0.0
precision: 0.6428571428571429
recall: 1.0
f1: 0.782608695652174
accuracy: 0.9624060150375939

Threshold 0.04
Model 1
far: 0.024193548387096774
frr: 0.1111111111111111
precision: 0.7272727272727273
recall: 0.8888888888888888
f1: 0.7999999999999999
accuracy: 0.9699248120300752

Threshold 0.05
Model 1
far: 0.016129032258064516
frr: 0.1111111111111111
precision: 0.8
recall: 0.8888888888888888
f1: 0.8421052631578948
accuracy: 0.9774436090225563

Threshold 0.06
Model 1
far: 0.0
frr: 0.1111111111111111
precision: 1.0
recall: 0.8888888888888888
f1: 0.9411764705882353
accuracy: 0.9924812030075187

Threshold 0.07
Model 1
far: 0.0
frr: 0.1111111111111

#### Conner1
Best Threshold 0.45 BOTH

In [24]:
username = usernames[8]
test_x = testing_xs[8]
test_y = testing_ys[8]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(1, 10):
    thresh = 0.1 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR CONNER1
Threshold 0.1
Model 1
far: 0.06451612903225806
frr: 0.0
precision: 0.4666666666666667
recall: 1.0
f1: 0.6363636363636364
accuracy: 0.9389312977099237

Threshold 0.2
Model 1
far: 0.016129032258064516
frr: 0.0
precision: 0.7777777777777778
recall: 1.0
f1: 0.8750000000000001
accuracy: 0.9847328244274809

Threshold 0.30000000000000004
Model 1
far: 0.008064516129032258
frr: 0.0
precision: 0.875
recall: 1.0
f1: 0.9333333333333333
accuracy: 0.9923664122137404

Threshold 0.4
Model 1
far: 0.0
frr: 0.0
precision: 1.0
recall: 1.0
f1: 1.0
accuracy: 1.0

Threshold 0.5
Model 1
far: 0.0
frr: 0.0
precision: 1.0
recall: 1.0
f1: 1.0
accuracy: 1.0

Threshold 0.6000000000000001
Model 1
far: 0.0
frr: 0.42857142857142855
precision: 1.0
recall: 0.5714285714285714
f1: 0.7272727272727273
accuracy: 0.9770992366412213

Threshold 0.7000000000000001
Model 1
far: 0.0
frr: 0.7142857142857143
precision: 1.0
recall: 0.2857142857142857
f1: 0.4444444444444445
accuracy: 0.9618320610687023

Threshold 

#### Cormac1
Best Threshold 0.6 Both

In [25]:
username = usernames[9]
test_x = testing_xs[9]
test_y = testing_ys[9]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(1, 10):
    thresh = 0.1 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR CORMAC1
Threshold 0.1
Model 1
far: 0.04838709677419355
frr: 0.0
precision: 0.5384615384615384
recall: 1.0
f1: 0.7000000000000001
accuracy: 0.9541984732824428

Threshold 0.2
Model 1
far: 0.04032258064516129
frr: 0.0
precision: 0.5833333333333334
recall: 1.0
f1: 0.7368421052631579
accuracy: 0.9618320610687023

Threshold 0.30000000000000004
Model 1
far: 0.04032258064516129
frr: 0.0
precision: 0.5833333333333334
recall: 1.0
f1: 0.7368421052631579
accuracy: 0.9618320610687023

Threshold 0.4
Model 1
far: 0.024193548387096774
frr: 0.0
precision: 0.7
recall: 1.0
f1: 0.8235294117647058
accuracy: 0.9770992366412213

Threshold 0.5
Model 1
far: 0.016129032258064516
frr: 0.0
precision: 0.7777777777777778
recall: 1.0
f1: 0.8750000000000001
accuracy: 0.9847328244274809

Threshold 0.6000000000000001
Model 1
far: 0.008064516129032258
frr: 0.0
precision: 0.875
recall: 1.0
f1: 0.9333333333333333
accuracy: 0.9923664122137404

Threshold 0.7000000000000001
Model 1
far: 0.008064516129032258
frr:

#### David1
Best Threshold EER: 0.02 F: 0.1

In [27]:
username = usernames[10]
test_x = testing_xs[10]
test_y = testing_ys[10]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(1, 10):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR DAVID1
Threshold 0.01
Model 1
far: 0.11290322580645161
frr: 0.1111111111111111
precision: 0.36363636363636365
recall: 0.8888888888888888
f1: 0.5161290322580644
accuracy: 0.8872180451127819

Threshold 0.02
Model 1
far: 0.056451612903225805
frr: 0.1111111111111111
precision: 0.5333333333333333
recall: 0.8888888888888888
f1: 0.6666666666666667
accuracy: 0.9398496240601504

Threshold 0.03
Model 1
far: 0.03225806451612903
frr: 0.1111111111111111
precision: 0.6666666666666666
recall: 0.8888888888888888
f1: 0.761904761904762
accuracy: 0.9624060150375939

Threshold 0.04
Model 1
far: 0.016129032258064516
frr: 0.1111111111111111
precision: 0.8
recall: 0.8888888888888888
f1: 0.8421052631578948
accuracy: 0.9774436090225563

Threshold 0.05
Model 1
far: 0.008064516129032258
frr: 0.1111111111111111
precision: 0.8888888888888888
recall: 0.8888888888888888
f1: 0.8888888888888888
accuracy: 0.9849624060150376

Threshold 0.06
Model 1
far: 0.008064516129032258
frr: 0.1111111111111111
precision

#### Graham1
Best Threshold 0.1 Both

In [28]:
username = usernames[11]
test_x = testing_xs[11]
test_y = testing_ys[11]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(1, 10):
    thresh = 0.1 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR GRAHAM1
Threshold 0.1
Model 1
far: 0.008064516129032258
frr: 0.0
precision: 0.9
recall: 1.0
f1: 0.9473684210526316
accuracy: 0.9924812030075187

Threshold 0.2
Model 1
far: 0.008064516129032258
frr: 0.1111111111111111
precision: 0.8888888888888888
recall: 0.8888888888888888
f1: 0.8888888888888888
accuracy: 0.9849624060150376

Threshold 0.30000000000000004
Model 1
far: 0.008064516129032258
frr: 0.3333333333333333
precision: 0.8571428571428571
recall: 0.6666666666666666
f1: 0.75
accuracy: 0.9699248120300752

Threshold 0.4
Model 1
far: 0.008064516129032258
frr: 0.3333333333333333
precision: 0.8571428571428571
recall: 0.6666666666666666
f1: 0.75
accuracy: 0.9699248120300752

Threshold 0.5
Model 1
far: 0.0
frr: 0.3333333333333333
precision: 1.0
recall: 0.6666666666666666
f1: 0.8
accuracy: 0.9774436090225563

Threshold 0.6000000000000001
Model 1
far: 0.0
frr: 0.5555555555555556
precision: 1.0
recall: 0.4444444444444444
f1: 0.6153846153846153
accuracy: 0.9624060150375939

Threshol

#### Hildigerr
Best Threshold 0.4 Both

In [30]:
username = usernames[12]
test_x = testing_xs[12]
test_y = testing_ys[12]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(35, 50):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR HILDIGERR
Threshold 0.35000000000000003
Model 1
far: 0.024193548387096774
frr: 0.0
precision: 0.7692307692307693
recall: 1.0
f1: 0.8695652173913044
accuracy: 0.9776119402985075

Threshold 0.36
Model 1
far: 0.016129032258064516
frr: 0.0
precision: 0.8333333333333334
recall: 1.0
f1: 0.9090909090909091
accuracy: 0.9850746268656716

Threshold 0.37
Model 1
far: 0.016129032258064516
frr: 0.0
precision: 0.8333333333333334
recall: 1.0
f1: 0.9090909090909091
accuracy: 0.9850746268656716

Threshold 0.38
Model 1
far: 0.016129032258064516
frr: 0.0
precision: 0.8333333333333334
recall: 1.0
f1: 0.9090909090909091
accuracy: 0.9850746268656716

Threshold 0.39
Model 1
far: 0.016129032258064516
frr: 0.0
precision: 0.8333333333333334
recall: 1.0
f1: 0.9090909090909091
accuracy: 0.9850746268656716

Threshold 0.4
Model 1
far: 0.016129032258064516
frr: 0.0
precision: 0.8333333333333334
recall: 1.0
f1: 0.9090909090909091
accuracy: 0.9850746268656716

Threshold 0.41000000000000003
Model 1
far: 0.

#### Ian1
Best Threshold 0.2 Both

In [31]:
username = usernames[13]
test_x = testing_xs[13]
test_y = testing_ys[13]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(1, 10):
    thresh = 0.1 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR IAN1
Threshold 0.1
Model 1
far: 0.08870967741935484
frr: 0.0
precision: 0.47619047619047616
recall: 1.0
f1: 0.6451612903225806
accuracy: 0.917910447761194

Threshold 0.2
Model 1
far: 0.04032258064516129
frr: 0.0
precision: 0.6666666666666666
recall: 1.0
f1: 0.8
accuracy: 0.9626865671641791

Threshold 0.30000000000000004
Model 1
far: 0.03225806451612903
frr: 0.1
precision: 0.6923076923076923
recall: 0.9
f1: 0.7826086956521738
accuracy: 0.9626865671641791

Threshold 0.4
Model 1
far: 0.008064516129032258
frr: 0.4
precision: 0.8571428571428571
recall: 0.6
f1: 0.7058823529411764
accuracy: 0.9626865671641791

Threshold 0.5
Model 1
far: 0.0
frr: 0.6
precision: 1.0
recall: 0.4
f1: 0.5714285714285715
accuracy: 0.9552238805970149

Threshold 0.6000000000000001
Model 1
far: 0.0
frr: 0.9
precision: 1.0
recall: 0.1
f1: 0.18181818181818182
accuracy: 0.9328358208955224

Threshold 0.7000000000000001
Model 1
far: 0.0
frr: 1.0
precision: -1
recall: 0.0
f1: 0.0
accuracy: 0.9253731343283582

T

#### Ian2
Best Threshold 0.09 Both

In [33]:
username = usernames[14]
test_x = testing_xs[14]
test_y = testing_ys[14]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(5, 20):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR IAN2
Threshold 0.05
Model 1
far: 0.0
frr: 0.0
precision: 1.0
recall: 1.0
f1: 1.0
accuracy: 1.0

Threshold 0.06
Model 1
far: 0.0
frr: 0.0
precision: 1.0
recall: 1.0
f1: 1.0
accuracy: 1.0

Threshold 0.07
Model 1
far: 0.0
frr: 0.0
precision: 1.0
recall: 1.0
f1: 1.0
accuracy: 1.0

Threshold 0.08
Model 1
far: 0.0
frr: 0.0
precision: 1.0
recall: 1.0
f1: 1.0
accuracy: 1.0

Threshold 0.09
Model 1
far: 0.0
frr: 0.0
precision: 1.0
recall: 1.0
f1: 1.0
accuracy: 1.0

Threshold 0.1
Model 1
far: 0.0
frr: 0.0
precision: 1.0
recall: 1.0
f1: 1.0
accuracy: 1.0

Threshold 0.11
Model 1
far: 0.0
frr: 0.0
precision: 1.0
recall: 1.0
f1: 1.0
accuracy: 1.0

Threshold 0.12
Model 1
far: 0.0
frr: 0.0
precision: 1.0
recall: 1.0
f1: 1.0
accuracy: 1.0

Threshold 0.13
Model 1
far: 0.0
frr: 0.125
precision: 1.0
recall: 0.875
f1: 0.9333333333333333
accuracy: 0.9924242424242424

Threshold 0.14
Model 1
far: 0.0
frr: 0.125
precision: 1.0
recall: 0.875
f1: 0.9333333333333333
accuracy: 0.9924242424242424

Thres

#### Jacob1
Best Threshold EER: 0.1  F: 0.2

In [35]:
username = usernames[15]
test_x = testing_xs[15]
test_y = testing_ys[15]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(5, 25):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR JACOB1
Threshold 0.05
Model 1
far: 0.10483870967741936
frr: 0.0
precision: 0.38095238095238093
recall: 1.0
f1: 0.5517241379310345
accuracy: 0.9015151515151515

Threshold 0.06
Model 1
far: 0.10483870967741936
frr: 0.0
precision: 0.38095238095238093
recall: 1.0
f1: 0.5517241379310345
accuracy: 0.9015151515151515

Threshold 0.07
Model 1
far: 0.08870967741935484
frr: 0.0
precision: 0.42105263157894735
recall: 1.0
f1: 0.5925925925925926
accuracy: 0.9166666666666666

Threshold 0.08
Model 1
far: 0.08064516129032258
frr: 0.0
precision: 0.4444444444444444
recall: 1.0
f1: 0.6153846153846153
accuracy: 0.9242424242424242

Threshold 0.09
Model 1
far: 0.08064516129032258
frr: 0.0
precision: 0.4444444444444444
recall: 1.0
f1: 0.6153846153846153
accuracy: 0.9242424242424242

Threshold 0.1
Model 1
far: 0.07258064516129033
frr: 0.125
precision: 0.4375
recall: 0.875
f1: 0.5833333333333334
accuracy: 0.9242424242424242

Threshold 0.11
Model 1
far: 0.07258064516129033
frr: 0.125
precision: 0.43

#### Jake1
Best Threshold 0.09 Both

In [37]:
username = usernames[16]
test_x = testing_xs[16]
test_y = testing_ys[16]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(5, 20):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR JAKE1
Threshold 0.05
Model 1
far: 0.12903225806451613
frr: 0.0
precision: 0.36
recall: 1.0
f1: 0.5294117647058824
accuracy: 0.8796992481203008

Threshold 0.06
Model 1
far: 0.12096774193548387
frr: 0.0
precision: 0.375
recall: 1.0
f1: 0.5454545454545454
accuracy: 0.8872180451127819

Threshold 0.07
Model 1
far: 0.12096774193548387
frr: 0.0
precision: 0.375
recall: 1.0
f1: 0.5454545454545454
accuracy: 0.8872180451127819

Threshold 0.08
Model 1
far: 0.11290322580645161
frr: 0.0
precision: 0.391304347826087
recall: 1.0
f1: 0.5625
accuracy: 0.8947368421052632

Threshold 0.09
Model 1
far: 0.10483870967741936
frr: 0.0
precision: 0.4090909090909091
recall: 1.0
f1: 0.5806451612903226
accuracy: 0.9022556390977443

Threshold 0.1
Model 1
far: 0.10483870967741936
frr: 0.0
precision: 0.4090909090909091
recall: 1.0
f1: 0.5806451612903226
accuracy: 0.9022556390977443

Threshold 0.11
Model 1
far: 0.10483870967741936
frr: 0.1111111111111111
precision: 0.38095238095238093
recall: 0.8888888888

#### James1 *Bad Results From This User*
Best Threshold 0.08 Both

In [40]:
username = usernames[17]
test_x = testing_xs[17]
test_y = testing_ys[17]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(1, 10):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR JAMES1
Threshold 0.01
Model 1
far: 0.9838709677419355
frr: 0.0
precision: 0.08270676691729323
recall: 1.0
f1: 0.1527777777777778
accuracy: 0.0962962962962963

Threshold 0.02
Model 1
far: 0.782258064516129
frr: 0.0
precision: 0.10185185185185185
recall: 1.0
f1: 0.18487394957983191
accuracy: 0.2814814814814815

Threshold 0.03
Model 1
far: 0.6048387096774194
frr: 0.0
precision: 0.12790697674418605
recall: 1.0
f1: 0.2268041237113402
accuracy: 0.4444444444444444

Threshold 0.04
Model 1
far: 0.4838709677419355
frr: 0.09090909090909091
precision: 0.14285714285714285
recall: 0.9090909090909091
f1: 0.24691358024691357
accuracy: 0.5481481481481482

Threshold 0.05
Model 1
far: 0.3790322580645161
frr: 0.18181818181818182
precision: 0.16071428571428573
recall: 0.8181818181818182
f1: 0.2686567164179105
accuracy: 0.6370370370370371

Threshold 0.06
Model 1
far: 0.2903225806451613
frr: 0.18181818181818182
precision: 0.2
recall: 0.8181818181818182
f1: 0.32142857142857145
accuracy: 0.7185185

#### Jamison1
Best Threshold EER: 0.05 F: 0.11

In [41]:
username = usernames[18]
test_x = testing_xs[18]
test_y = testing_ys[18]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(1, 16):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR JAMISON1
Threshold 0.01
Model 1
far: 0.0967741935483871
frr: 0.0
precision: 0.45454545454545453
recall: 1.0
f1: 0.625
accuracy: 0.9104477611940298

Threshold 0.02
Model 1
far: 0.06451612903225806
frr: 0.0
precision: 0.5555555555555556
recall: 1.0
f1: 0.7142857142857143
accuracy: 0.9402985074626866

Threshold 0.03
Model 1
far: 0.04032258064516129
frr: 0.1
precision: 0.6428571428571429
recall: 0.9
f1: 0.75
accuracy: 0.9552238805970149

Threshold 0.04
Model 1
far: 0.03225806451612903
frr: 0.1
precision: 0.6923076923076923
recall: 0.9
f1: 0.7826086956521738
accuracy: 0.9626865671641791

Threshold 0.05
Model 1
far: 0.03225806451612903
frr: 0.1
precision: 0.6923076923076923
recall: 0.9
f1: 0.7826086956521738
accuracy: 0.9626865671641791

Threshold 0.06
Model 1
far: 0.03225806451612903
frr: 0.1
precision: 0.6923076923076923
recall: 0.9
f1: 0.7826086956521738
accuracy: 0.9626865671641791

Threshold 0.07
Model 1
far: 0.03225806451612903
frr: 0.1
precision: 0.6923076923076923
recall

#### John1
Best Threshold EER:0.35 F: 0.5

In [55]:
username = usernames[19]
test_x = testing_xs[19]
test_y = testing_ys[19]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(30, 40):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR JOHN1
Threshold 0.3
Model 1
far: 0.12096774193548387
frr: 0.0
precision: 0.375
recall: 1.0
f1: 0.5454545454545454
accuracy: 0.8872180451127819

Threshold 0.31
Model 1
far: 0.12096774193548387
frr: 0.0
precision: 0.375
recall: 1.0
f1: 0.5454545454545454
accuracy: 0.8872180451127819

Threshold 0.32
Model 1
far: 0.12096774193548387
frr: 0.0
precision: 0.375
recall: 1.0
f1: 0.5454545454545454
accuracy: 0.8872180451127819

Threshold 0.33
Model 1
far: 0.12096774193548387
frr: 0.1111111111111111
precision: 0.34782608695652173
recall: 0.8888888888888888
f1: 0.5
accuracy: 0.8796992481203008

Threshold 0.34
Model 1
far: 0.11290322580645161
frr: 0.1111111111111111
precision: 0.36363636363636365
recall: 0.8888888888888888
f1: 0.5161290322580644
accuracy: 0.8872180451127819

Threshold 0.35000000000000003
Model 1
far: 0.11290322580645161
frr: 0.1111111111111111
precision: 0.36363636363636365
recall: 0.8888888888888888
f1: 0.5161290322580644
accuracy: 0.8872180451127819

Threshold 0.36
M

#### Jonah1
Best Threshold 0.37 Both

In [56]:
username = usernames[20]
test_x = testing_xs[20]
test_y = testing_ys[20]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(30, 45):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR JONAH1
Threshold 0.3
Model 1
far: 0.03225806451612903
frr: 0.0
precision: 0.6923076923076923
recall: 1.0
f1: 0.8181818181818181
accuracy: 0.9699248120300752

Threshold 0.31
Model 1
far: 0.024193548387096774
frr: 0.0
precision: 0.75
recall: 1.0
f1: 0.8571428571428571
accuracy: 0.9774436090225563

Threshold 0.32
Model 1
far: 0.024193548387096774
frr: 0.0
precision: 0.75
recall: 1.0
f1: 0.8571428571428571
accuracy: 0.9774436090225563

Threshold 0.33
Model 1
far: 0.024193548387096774
frr: 0.0
precision: 0.75
recall: 1.0
f1: 0.8571428571428571
accuracy: 0.9774436090225563

Threshold 0.34
Model 1
far: 0.016129032258064516
frr: 0.0
precision: 0.8181818181818182
recall: 1.0
f1: 0.9
accuracy: 0.9849624060150376

Threshold 0.35000000000000003
Model 1
far: 0.016129032258064516
frr: 0.0
precision: 0.8181818181818182
recall: 1.0
f1: 0.9
accuracy: 0.9849624060150376

Threshold 0.36
Model 1
far: 0.008064516129032258
frr: 0.0
precision: 0.9
recall: 1.0
f1: 0.9473684210526316
accuracy: 0.9

#### Joseph1
Best Threshold EER: 0.06 F: 0.08

In [57]:
username = usernames[21]
test_x = testing_xs[21]
test_y = testing_ys[21]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(1, 10):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR JOSEPH1
Threshold 0.01
Model 1
far: 0.8467741935483871
frr: 0.0
precision: 0.07079646017699115
recall: 1.0
f1: 0.13223140495867766
accuracy: 0.20454545454545456

Threshold 0.02
Model 1
far: 0.5080645161290323
frr: 0.0
precision: 0.11267605633802817
recall: 1.0
f1: 0.20253164556962025
accuracy: 0.5227272727272727

Threshold 0.03
Model 1
far: 0.27419354838709675
frr: 0.0
precision: 0.19047619047619047
recall: 1.0
f1: 0.32
accuracy: 0.7424242424242424

Threshold 0.04
Model 1
far: 0.18548387096774194
frr: 0.125
precision: 0.23333333333333334
recall: 0.875
f1: 0.3684210526315789
accuracy: 0.8181818181818182

Threshold 0.05
Model 1
far: 0.13709677419354838
frr: 0.125
precision: 0.2916666666666667
recall: 0.875
f1: 0.43750000000000006
accuracy: 0.8636363636363636

Threshold 0.06
Model 1
far: 0.10483870967741936
frr: 0.125
precision: 0.35
recall: 0.875
f1: 0.4999999999999999
accuracy: 0.8939393939393939

Threshold 0.07
Model 1
far: 0.07258064516129033
frr: 0.25
precision: 0.4
reca

#### Kevin1
Best Threshold 0.38 Both

In [58]:
username = usernames[22]
test_x = testing_xs[22]
test_y = testing_ys[22]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(30, 40):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR KEVIN1
Threshold 0.3
Model 1
far: 0.016129032258064516
frr: 0.1111111111111111
precision: 0.8
recall: 0.8888888888888888
f1: 0.8421052631578948
accuracy: 0.9774436090225563

Threshold 0.31
Model 1
far: 0.016129032258064516
frr: 0.1111111111111111
precision: 0.8
recall: 0.8888888888888888
f1: 0.8421052631578948
accuracy: 0.9774436090225563

Threshold 0.32
Model 1
far: 0.016129032258064516
frr: 0.1111111111111111
precision: 0.8
recall: 0.8888888888888888
f1: 0.8421052631578948
accuracy: 0.9774436090225563

Threshold 0.33
Model 1
far: 0.016129032258064516
frr: 0.1111111111111111
precision: 0.8
recall: 0.8888888888888888
f1: 0.8421052631578948
accuracy: 0.9774436090225563

Threshold 0.34
Model 1
far: 0.016129032258064516
frr: 0.1111111111111111
precision: 0.8
recall: 0.8888888888888888
f1: 0.8421052631578948
accuracy: 0.9774436090225563

Threshold 0.35000000000000003
Model 1
far: 0.016129032258064516
frr: 0.1111111111111111
precision: 0.8
recall: 0.8888888888888888
f1: 0.84210

#### Koji1
Best Threshold 0.1 Both

In [60]:
username = usernames[23]
test_x = testing_xs[23]
test_y = testing_ys[23]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(1, 15):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR KOJI11
Threshold 0.01
Model 1
far: 0.07258064516129033
frr: 0.0
precision: 0.5263157894736842
recall: 1.0
f1: 0.6896551724137931
accuracy: 0.9328358208955224

Threshold 0.02
Model 1
far: 0.056451612903225805
frr: 0.0
precision: 0.5882352941176471
recall: 1.0
f1: 0.7407407407407407
accuracy: 0.9477611940298507

Threshold 0.03
Model 1
far: 0.04838709677419355
frr: 0.0
precision: 0.625
recall: 1.0
f1: 0.7692307692307693
accuracy: 0.9552238805970149

Threshold 0.04
Model 1
far: 0.04838709677419355
frr: 0.0
precision: 0.625
recall: 1.0
f1: 0.7692307692307693
accuracy: 0.9552238805970149

Threshold 0.05
Model 1
far: 0.04838709677419355
frr: 0.0
precision: 0.625
recall: 1.0
f1: 0.7692307692307693
accuracy: 0.9552238805970149

Threshold 0.06
Model 1
far: 0.04838709677419355
frr: 0.0
precision: 0.625
recall: 1.0
f1: 0.7692307692307693
accuracy: 0.9552238805970149

Threshold 0.07
Model 1
far: 0.04838709677419355
frr: 0.0
precision: 0.625
recall: 1.0
f1: 0.7692307692307693
accuracy: 

#### Leon1
Best Threshold 0.1 Both

In [61]:
username = usernames[24]
test_x = testing_xs[24]
test_y = testing_ys[24]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(5, 20):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR LEON1
Threshold 0.05
Model 1
far: 0.016129032258064516
frr: 0.0
precision: 0.8461538461538461
recall: 1.0
f1: 0.9166666666666666
accuracy: 0.9851851851851852

Threshold 0.06
Model 1
far: 0.016129032258064516
frr: 0.0
precision: 0.8461538461538461
recall: 1.0
f1: 0.9166666666666666
accuracy: 0.9851851851851852

Threshold 0.07
Model 1
far: 0.016129032258064516
frr: 0.0
precision: 0.8461538461538461
recall: 1.0
f1: 0.9166666666666666
accuracy: 0.9851851851851852

Threshold 0.08
Model 1
far: 0.016129032258064516
frr: 0.0
precision: 0.8461538461538461
recall: 1.0
f1: 0.9166666666666666
accuracy: 0.9851851851851852

Threshold 0.09
Model 1
far: 0.016129032258064516
frr: 0.0
precision: 0.8461538461538461
recall: 1.0
f1: 0.9166666666666666
accuracy: 0.9851851851851852

Threshold 0.1
Model 1
far: 0.016129032258064516
frr: 0.0
precision: 0.8461538461538461
recall: 1.0
f1: 0.9166666666666666
accuracy: 0.9851851851851852

Threshold 0.11
Model 1
far: 0.016129032258064516
frr: 0.0
precis

#### Michael Mosin1 *Poor Results - Not horrible*
Best Threshold EER: 0.04 F; 0.07

In [62]:
username = usernames[25]
test_x = testing_xs[25]
test_y = testing_ys[25]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(1, 10):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR MICHAEL MOSIN1
Threshold 0.01
Model 1
far: 0.6451612903225806
frr: 0.0
precision: 0.1111111111111111
recall: 1.0
f1: 0.19999999999999998
accuracy: 0.40298507462686567

Threshold 0.02
Model 1
far: 0.4435483870967742
frr: 0.1
precision: 0.140625
recall: 0.9
f1: 0.24324324324324326
accuracy: 0.582089552238806

Threshold 0.03
Model 1
far: 0.25806451612903225
frr: 0.1
precision: 0.21951219512195122
recall: 0.9
f1: 0.3529411764705882
accuracy: 0.753731343283582

Threshold 0.04
Model 1
far: 0.21774193548387097
frr: 0.2
precision: 0.22857142857142856
recall: 0.8
f1: 0.3555555555555555
accuracy: 0.7835820895522388

Threshold 0.05
Model 1
far: 0.1693548387096774
frr: 0.3
precision: 0.25
recall: 0.7
f1: 0.3684210526315789
accuracy: 0.8208955223880597

Threshold 0.06
Model 1
far: 0.12903225806451613
frr: 0.3
precision: 0.30434782608695654
recall: 0.7
f1: 0.42424242424242425
accuracy: 0.8582089552238806

Threshold 0.07
Model 1
far: 0.10483870967741936
frr: 0.3
precision: 0.35
recall: 0

#### Mike1
Best Threshold EER: 0.35 F: 0.33

In [63]:
username = usernames[26]
test_x = testing_xs[26]
test_y = testing_ys[26]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(30, 40):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR MIKE1
Threshold 0.3
Model 1
far: 0.1935483870967742
frr: 0.0
precision: 0.29411764705882354
recall: 1.0
f1: 0.45454545454545453
accuracy: 0.8208955223880597

Threshold 0.31
Model 1
far: 0.1774193548387097
frr: 0.0
precision: 0.3125
recall: 1.0
f1: 0.47619047619047616
accuracy: 0.835820895522388

Threshold 0.32
Model 1
far: 0.1693548387096774
frr: 0.0
precision: 0.3225806451612903
recall: 1.0
f1: 0.4878048780487805
accuracy: 0.8432835820895522

Threshold 0.33
Model 1
far: 0.16129032258064516
frr: 0.0
precision: 0.3333333333333333
recall: 1.0
f1: 0.5
accuracy: 0.8507462686567164

Threshold 0.34
Model 1
far: 0.1532258064516129
frr: 0.1
precision: 0.32142857142857145
recall: 0.9
f1: 0.4736842105263158
accuracy: 0.8507462686567164

Threshold 0.35000000000000003
Model 1
far: 0.1532258064516129
frr: 0.1
precision: 0.32142857142857145
recall: 0.9
f1: 0.4736842105263158
accuracy: 0.8507462686567164

Threshold 0.36
Model 1
far: 0.14516129032258066
frr: 0.2
precision: 0.3076923076923

#### Phil1
Best Threshold EER: 0.38  F: 0.5

In [65]:
username = usernames[27]
test_x = testing_xs[27]
test_y = testing_ys[27]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(35, 51):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR PHIL1
Threshold 0.35000000000000003
Model 1
far: 0.056451612903225805
frr: 0.0
precision: 0.5
recall: 1.0
f1: 0.6666666666666666
accuracy: 0.9465648854961832

Threshold 0.36
Model 1
far: 0.04838709677419355
frr: 0.0
precision: 0.5384615384615384
recall: 1.0
f1: 0.7000000000000001
accuracy: 0.9541984732824428

Threshold 0.37
Model 1
far: 0.04032258064516129
frr: 0.0
precision: 0.5833333333333334
recall: 1.0
f1: 0.7368421052631579
accuracy: 0.9618320610687023

Threshold 0.38
Model 1
far: 0.04032258064516129
frr: 0.0
precision: 0.5833333333333334
recall: 1.0
f1: 0.7368421052631579
accuracy: 0.9618320610687023

Threshold 0.39
Model 1
far: 0.04032258064516129
frr: 0.0
precision: 0.5833333333333334
recall: 1.0
f1: 0.7368421052631579
accuracy: 0.9618320610687023

Threshold 0.4
Model 1
far: 0.04032258064516129
frr: 0.14285714285714285
precision: 0.5454545454545454
recall: 0.8571428571428571
f1: 0.6666666666666665
accuracy: 0.9541984732824428

Threshold 0.41000000000000003
Model 1


#### Reid1
Best Threshold EER: 0.35 F: 0.43

In [66]:
username = usernames[28]
test_x = testing_xs[28]
test_y = testing_ys[28]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(30, 50):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR REID1
Threshold 0.3
Model 1
far: 0.16129032258064516
frr: 0.0
precision: 0.25925925925925924
recall: 1.0
f1: 0.4117647058823529
accuracy: 0.8473282442748091

Threshold 0.31
Model 1
far: 0.14516129032258066
frr: 0.0
precision: 0.28
recall: 1.0
f1: 0.43750000000000006
accuracy: 0.8625954198473282

Threshold 0.32
Model 1
far: 0.14516129032258066
frr: 0.0
precision: 0.28
recall: 1.0
f1: 0.43750000000000006
accuracy: 0.8625954198473282

Threshold 0.33
Model 1
far: 0.12903225806451613
frr: 0.0
precision: 0.30434782608695654
recall: 1.0
f1: 0.4666666666666667
accuracy: 0.8778625954198473

Threshold 0.34
Model 1
far: 0.12903225806451613
frr: 0.0
precision: 0.30434782608695654
recall: 1.0
f1: 0.4666666666666667
accuracy: 0.8778625954198473

Threshold 0.35000000000000003
Model 1
far: 0.11290322580645161
frr: 0.0
precision: 0.3333333333333333
recall: 1.0
f1: 0.5
accuracy: 0.8931297709923665

Threshold 0.36
Model 1
far: 0.10483870967741936
frr: 0.14285714285714285
precision: 0.3157894

#### SamP1
Best Threshold EER: 0.1 F: 0.4

In [52]:
username = usernames[29]
test_x = testing_xs[29]
test_y = testing_ys[29]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(1, 10):
    thresh = 0.1 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR SAMP1
Threshold 0.1
Model 1
far: 0.06451612903225806
frr: 0.0
precision: 0.5294117647058824
recall: 1.0
f1: 0.6923076923076924
accuracy: 0.9398496240601504

Threshold 0.2
Model 1
far: 0.04838709677419355
frr: 0.1111111111111111
precision: 0.5714285714285714
recall: 0.8888888888888888
f1: 0.6956521739130435
accuracy: 0.9473684210526315

Threshold 0.30000000000000004
Model 1
far: 0.024193548387096774
frr: 0.1111111111111111
precision: 0.7272727272727273
recall: 0.8888888888888888
f1: 0.7999999999999999
accuracy: 0.9699248120300752

Threshold 0.4
Model 1
far: 0.016129032258064516
frr: 0.1111111111111111
precision: 0.8
recall: 0.8888888888888888
f1: 0.8421052631578948
accuracy: 0.9774436090225563

Threshold 0.5
Model 1
far: 0.008064516129032258
frr: 0.2222222222222222
precision: 0.875
recall: 0.7777777777777778
f1: 0.823529411764706
accuracy: 0.9774436090225563

Threshold 0.6000000000000001
Model 1
far: 0.008064516129032258
frr: 0.3333333333333333
precision: 0.8571428571428571

#### Sean1
Best Threshold 0.4 Both

In [68]:
username = usernames[30]
test_x = testing_xs[30]
test_y = testing_ys[30]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(40, 51):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR SEAN1
Threshold 0.4
Model 1
far: 0.03225806451612903
frr: 0.0
precision: 0.7333333333333333
recall: 1.0
f1: 0.846153846153846
accuracy: 0.9703703703703703

Threshold 0.41000000000000003
Model 1
far: 0.03225806451612903
frr: 0.0
precision: 0.7333333333333333
recall: 1.0
f1: 0.846153846153846
accuracy: 0.9703703703703703

Threshold 0.42
Model 1
far: 0.03225806451612903
frr: 0.09090909090909091
precision: 0.7142857142857143
recall: 0.9090909090909091
f1: 0.8
accuracy: 0.9629629629629629

Threshold 0.43
Model 1
far: 0.03225806451612903
frr: 0.09090909090909091
precision: 0.7142857142857143
recall: 0.9090909090909091
f1: 0.8
accuracy: 0.9629629629629629

Threshold 0.44
Model 1
far: 0.024193548387096774
frr: 0.09090909090909091
precision: 0.7692307692307693
recall: 0.9090909090909091
f1: 0.8333333333333333
accuracy: 0.9703703703703703

Threshold 0.45
Model 1
far: 0.024193548387096774
frr: 0.09090909090909091
precision: 0.7692307692307693
recall: 0.9090909090909091
f1: 0.83333333

#### Theo1
Best Threshold EER: 0.1 F: 0.35

In [69]:
username = usernames[31]
test_x = testing_xs[31]
test_y = testing_ys[31]

path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
filename = username + '_model.h5'
userfile = os.path.join(path, filename)
clf = load_model(userfile)
    
print(f'ACCURACY FOR {username.upper()}')
for i in range(30, 40):
    thresh = 0.01 * i
    y_preds = test_batches(clf, test_x, thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    print(f'Threshold {thresh}')
    print('Model 1')
    for k, v in performance.items():
        print(f'{k}: {v}')
    print()


ACCURACY FOR THEO1
Threshold 0.3
Model 1
far: 0.016129032258064516
frr: 0.16666666666666666
precision: 0.8333333333333334
recall: 0.8333333333333334
f1: 0.8333333333333334
accuracy: 0.9705882352941176

Threshold 0.31
Model 1
far: 0.016129032258064516
frr: 0.16666666666666666
precision: 0.8333333333333334
recall: 0.8333333333333334
f1: 0.8333333333333334
accuracy: 0.9705882352941176

Threshold 0.32
Model 1
far: 0.016129032258064516
frr: 0.16666666666666666
precision: 0.8333333333333334
recall: 0.8333333333333334
f1: 0.8333333333333334
accuracy: 0.9705882352941176

Threshold 0.33
Model 1
far: 0.016129032258064516
frr: 0.16666666666666666
precision: 0.8333333333333334
recall: 0.8333333333333334
f1: 0.8333333333333334
accuracy: 0.9705882352941176

Threshold 0.34
Model 1
far: 0.016129032258064516
frr: 0.16666666666666666
precision: 0.8333333333333334
recall: 0.8333333333333334
f1: 0.8333333333333334
accuracy: 0.9705882352941176

Threshold 0.35000000000000003
Model 1
far: 0.00806451612903225

## Testing Overall Accuracy

In [70]:
eer_thresholds = {
    'Amanda11': 0.16,
    'Amelia1': 0.15,
    'Andy1': 0.1,
    'Andy21': 0.009,
    'Benjamin1': 0.16, 
    'Blair1': 0.05,
    'Blake1': 0.02, 
    'Chloe1': 0.03, 
    'Conner1': 0.45, 
    'Cormac1': 0.6, 
    'David1': 0.02,
    'Graham1': 0.1,
    'Hildigerr': 0.4,
    'Ian1': 0.2, 
    'Ian2': 0.09, 
    'Jacob1': 0.1, 
    'Jake1': 0.09,
    'James1': 0.08,
    'Jamison1': 0.05, 
    'John1': 0.35, 
    'Jonah1': 0.37, 
    'Joseph1': 0.06,
    'Kevin1': 0.38,
    'Koji11': 0.1,
    'Leon1': 0.1,
    'Michael Mosin1': 0.04,
    'Mike1': 0.35,
    'Phil1': 0.38,
    'Reid1': 0.35,
    'SamP1': 0.1, 
    'Sean1': 0.4,
    'Theo1': 0.1    
}


f_thresholds = {
    'Amanda11': 0.2,
    'Amelia1': 0.15,
    'Andy1': 0.73,
    'Andy21': 0.009,
    'Benjamin1': 0.3, 
    'Blair1': 0.05,
    'Blake1': 0.02, 
    'Chloe1': 0.07, 
    'Conner1': 0.45, 
    'Cormac1': 0.6, 
    'David1': 0.1,
    'Graham1': 0.1,
    'Hildigerr': 0.4,
    'Ian1': 0.2, 
    'Ian2': 0.09, 
    'Jacob1': 0.2, 
    'Jake1': 0.09,
    'James1': 0.08,
    'Jamison1': 0.11, 
    'John1': 0.5, 
    'Jonah1': 0.37, 
    'Joseph1': 0.08,
    'Kevin1': 0.38,
    'Koji11': 0.1,
    'Leon1': 0.1,
    'Michael Mosin1': 0.07,
    'Mike1': 0.33,
    'Phil1': 0.5,
    'Reid1': 0.43,
    'SamP1': 0.4, 
    'Sean1': 0.4,
    'Theo1': 0.35    
}


eer_dicts = []
f_dicts = []
    
for i in range(len(usernames)):
    username = usernames[i]
    eer_thresh = eer_thresholds[username]
    f_thresh = f_thresholds[username]
    test_x = testing_xs[i]
    test_y = testing_ys[i]
    filename = username + '_model.h5'
    path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
    userfile = os.path.join(path, filename)
    clf = load_model(userfile)
    y_preds = test_batches(clf, test_x, eer_thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    eer_dicts.append(performance)
    
    y_preds = test_batches(clf, test_x, f_thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    f_dicts.append(performance)

    
print('EER Performance')
test_overall_acc(eer_dicts)
print()
print('F-Score Performance')
test_overall_acc(f_dicts)


EER Performance
Overall Accuracy - Average of Individual Models:

FAR : 0.05771169354838709
FRR : 0.052049512987012984
Precision : 0.6125820640756302
Recall : 0.947950487012987
F1-Score : 0.7230259253850732
Accuracy : 0.9424984591759711

F-Score Performance
Overall Accuracy - Average of Individual Models:

FAR : 0.03679435483870967
FRR : 0.09330695346320346
Precision : 0.7197541607697858
Recall : 0.9066930465367965
F1-Score : 0.7815348314009511
Accuracy : 0.9594459588145814


## Test on All Data

In [71]:
# Get the Full Batches

test_path = os.path.join(os.getcwd(), 'RNN-Test-Aligned')
model_save_path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
test_files = os.listdir(test_path)

usernames = get_usernames(test_files)
test_dfs = getdfs(test_path, test_files)

testing_xs = []
testing_ys = []
for user in usernames:
    xtest, ytest = create_full_test(user, usernames, test_dfs)
    testing_xs.append(xtest)
    testing_ys.append(ytest)

In [72]:
eer_thresholds = {
    'Amanda11': 0.16,
    'Amelia1': 0.15,
    'Andy1': 0.1,
    'Andy21': 0.009,
    'Benjamin1': 0.16, 
    'Blair1': 0.05,
    'Blake1': 0.02, 
    'Chloe1': 0.03, 
    'Conner1': 0.45, 
    'Cormac1': 0.6, 
    'David1': 0.02,
    'Graham1': 0.1,
    'Hildigerr': 0.4,
    'Ian1': 0.2, 
    'Ian2': 0.09, 
    'Jacob1': 0.1, 
    'Jake1': 0.09,
    'James1': 0.08,
    'Jamison1': 0.05, 
    'John1': 0.35, 
    'Jonah1': 0.37, 
    'Joseph1': 0.06,
    'Kevin1': 0.38,
    'Koji11': 0.1,
    'Leon1': 0.1,
    'Michael Mosin1': 0.04,
    'Mike1': 0.35,
    'Phil1': 0.38,
    'Reid1': 0.35,
    'SamP1': 0.1, 
    'Sean1': 0.4,
    'Theo1': 0.1    
}


f_thresholds = {
    'Amanda11': 0.2,
    'Amelia1': 0.15,
    'Andy1': 0.73,
    'Andy21': 0.009,
    'Benjamin1': 0.3, 
    'Blair1': 0.05,
    'Blake1': 0.02, 
    'Chloe1': 0.07, 
    'Conner1': 0.45, 
    'Cormac1': 0.6, 
    'David1': 0.1,
    'Graham1': 0.1,
    'Hildigerr': 0.4,
    'Ian1': 0.2, 
    'Ian2': 0.09, 
    'Jacob1': 0.2, 
    'Jake1': 0.09,
    'James1': 0.08,
    'Jamison1': 0.11, 
    'John1': 0.5, 
    'Jonah1': 0.37, 
    'Joseph1': 0.08,
    'Kevin1': 0.38,
    'Koji11': 0.1,
    'Leon1': 0.1,
    'Michael Mosin1': 0.07,
    'Mike1': 0.33,
    'Phil1': 0.5,
    'Reid1': 0.43,
    'SamP1': 0.4, 
    'Sean1': 0.4,
    'Theo1': 0.35    
}


eer_dicts = []
f_dicts = []
    
for i in range(len(usernames)):
    username = usernames[i]
    eer_thresh = eer_thresholds[username]
    f_thresh = f_thresholds[username]
    test_x = testing_xs[i]
    test_y = testing_ys[i]
    filename = username + '_model.h5'
    path = os.path.join(os.getcwd(), 'RNNv4-saved_models')
    userfile = os.path.join(path, filename)
    clf = load_model(userfile)
    y_preds = test_batches(clf, test_x, eer_thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    eer_dicts.append(performance)
    
    y_preds = test_batches(clf, test_x, f_thresh)
    performance = calc_accuracy_metrics(y_preds, test_y)
    f_dicts.append(performance)

    
print('EER Performance')
test_overall_acc(eer_dicts)
print()
print('F-Score Performance')
test_overall_acc(f_dicts)

EER Performance
Overall Accuracy - Average of Individual Models:

FAR : 0.06751835791588423
FRR : 0.052049512987012984
Precision : 0.40131519844064273
Recall : 0.947950487012987
F1-Score : 0.5373209459225071
Accuracy : 0.9328947368421052

F-Score Performance
Overall Accuracy - Average of Individual Models:

FAR : 0.046246413047607536
FRR : 0.09330695346320346
Precision : 0.47702871748506886
Recall : 0.9066930465367965
F1-Score : 0.6012029057355459
Accuracy : 0.952302631578947


## Findings

With this model we acheived extremely good results on the 32-user test with performance just slightly under the 15-user test showing that this model scales well to more users where the prior work failed to do so to the same degree of accuracy. 

On all data:

When minimizing EER the FAR was 0.0675 and FRR was 0.0520 with an F1-Score of 0.537

When maximizing the F1-Score the FAR was 0.0462 and FRR was 0.0933 with an F1-Score of 0.601 and accuracy of 95.2%. 


On a subset of the data:

When minimizing EER the FAR was 0.0577 and FRR was 0.0520 with an F1-Score of 0.723

When maximizing the F1-Score the FAR was 0.0367 and FRR was 0.0933 with an F1-Score of 0.782 and accuracy of 95.9%.